<a href="https://colab.research.google.com/github/Sagi1977/CycleTrading-/blob/main/Cycle_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *** Base APP !!!***

In [59]:
# Install necessary libraries
!pip install pandas_ta
import yfinance as yf
import pandas as pd
import pandas_ta as ta

# --- 1. Define Assets ---
ASSET_GROUPS = {
    "ETF": ['SPY', 'QQQ', 'MAGS', 'IBIT', 'ETHA'],
    "STOCK": ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'NVDA', 'TSLA']
}

# --- 2. Define Trading Days (Adjusted for Short/Medium Term) ---
N_DAYS_SELECTED = 150 # Focusing on short to medium term trends

# --- 3. Function to Fetch Base Data ---
def get_base_data(ticker, days):
    """
    Fetches historical data (Open, Low, High, Close, Volume) for a given ticker.
    """
    try:
        # Fetch enough data to ensure the selected number of days are available
        # Fetching 1 year should be sufficient for 150 days
        df = yf.download(ticker, period="1y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # Keep only the required columns
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df_final = df[['Open', 'Low', 'High', 'Close', 'Volume']].tail(days)

    return df_final

# --- 4. Fetch Data and Calculate Indicators ---
fetched_data = {}
analyzed_data = {}

print("=========================================================")
print(f"        Fetching Data and Calculating Indicators ({N_DAYS_SELECTED} days)")
print("=========================================================")

for group_name, tickers in ASSET_GROUPS.items():
    print(f"\n################## Group: {group_name} ##################")
    fetched_data[group_name] = {}
    analyzed_data[group_name] = {}

    for ticker in tickers:
        print(f"\n--- Processing data for {ticker} ---")

        # Fetch data
        df_base = get_base_data(ticker, N_DAYS_SELECTED)

        if not df_base.empty:
            fetched_data[group_name][ticker] = df_base.copy() # Store fetched data

            # Calculate technical indicators using pandas_ta (Relevant for short/medium term)
            print(f"--- Calculating indicators for {ticker} ---")
            df_analysis = df_base.copy() # Work on a copy
            df_analysis.ta.rsi(length=14, append=True)
            df_analysis.ta.atr(length=14, append=True)
            df_analysis.ta.sma(length=50, append=True) # Medium-term MA
            df_analysis.ta.sma(length=100, append=True) # Longer end of medium-term MA
            df_analysis.ta.macd(fast=12, slow=26, signal=9, append=True)

            # Add more indicators (examples)
            df_analysis.ta.bbands(length=20, append=True) # Bollinger Bands
            df_analysis.ta.ema(length=20, append=True) # Exponential Moving Average (Short-term)
            df_analysis.ta.vwap(append=True) # Volume Weighted Average Price

            # Add Stochastic Oscillator with specified parameters (Short-term)
            stoch_k, stoch_d, stoch_smooth_k = 14, 3, 3
            df_analysis.ta.stoch(k=stoch_k, d=stoch_d, smooth_k=stoch_smooth_k, append=True)

            # --- Add Logic to Combine Stochastic with MAs (Adjusted for Medium Term) ---
            # Example: Check if Stochastic %K is above SMA 50 (Potential bullish signal relative to medium trend)
            stoch_k_col = f'STOCHk_{stoch_k}_{stoch_d}_{stoch_smooth_k}'
            sma_50_col = 'SMA_50'
            if stoch_k_col in df_analysis.columns and sma_50_col in df_analysis.columns:
                df_analysis['STOCHk_vs_SMA50_Above'] = (df_analysis[stoch_k_col] > df_analysis[sma_50_col]).astype(int)

            # Removed comparison to SMA 200 as we are focusing on shorter/medium terms

            # --- Add Fibonacci Retracement and Extension Levels ---
            # Calculate based on the High and Low of the current data range (150 days)
            fib_max_price = df_analysis['High'].max()
            fib_min_price = df_analysis['Low'].min()
            fib_range = fib_max_price - fib_min_price

            # Fibonacci Retracement Levels (from top to bottom for uptrend, or bottom to top for downtrend)
            # Common Retracement levels: 0, 23.6%, 38.2%, 50%, 61.8%, 78.6%, 100%
            df_analysis['Fib_0_Retracement'] = fib_max_price
            df_analysis['Fib_23.6_Retracement'] = fib_max_price - (fib_range * 0.236)
            df_analysis['Fib_38.2_Retracement'] = fib_max_price - (fib_range * 0.382)
            df_analysis['Fib_50.0_Retracement'] = fib_max_price - (fib_range * 0.50)
            df_analysis['Fib_61.8_Retracement'] = fib_max_price - (fib_range * 0.618)
            df_analysis['Fib_78.6_Retracement'] = fib_max_price - (fib_range * 0.786)
            df_analysis['Fib_100_Retracement'] = fib_min_price

            # Fibonacci Extension Levels (common levels projected from the range)
            # Common Extension levels: 123.6%, 138.2%, 161.8%, 200%, 261.8%
            # These are often projected from a swing low/high after a retracement.
            # For simplicity here, we project from the original range start/end.
            df_analysis['Fib_123.6_Extension'] = fib_max_price + (fib_range * 0.236)
            df_analysis['Fib_138.2_Extension'] = fib_max_price + (fib_range * 0.382)
            df_analysis['Fib_161.8_Extension'] = fib_max_price + (fib_range * 0.618)
            df_analysis['Fib_200.0_Extension'] = fib_max_price + (fib_range * 1.00)
            df_analysis['Fib_261.8_Extension'] = fib_max_price + (fib_range * 1.618)


            analyzed_data[group_name][ticker] = df_analysis # Store analyzed data
            print(f"Indicators calculated for {ticker}. DataFrame shape: {df_analysis.shape}")
        else:
            print(f"!!! Skipping {ticker} as no data was available.")

print("\n=========================================================")
print("        Data Fetching and Indicator Calculation Complete ")
print("=========================================================")

# --- 5. Save Analyzed Data to Excel ---
output_filename = "analyzed_financial_data.xlsx"

print(f"\n=========================================================")
print(f"        Saving Analyzed Data to {output_filename}         ")
print("=========================================================")

with pd.ExcelWriter(output_filename) as writer:
    for group_name, group_data in analyzed_data.items():
        if group_data: # Check if there is data for the group
            # Combine all tickers within the group into a single DataFrame for saving
            # Add a 'Ticker' column to identify the data for each ticker
            combined_df = pd.concat(
                [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
            )

            # Save the combined DataFrame for the group to a sheet named after the group
            combined_df.to_excel(writer, sheet_name=group_name)

            print(f"Analyzed data for {group_name} saved to sheet '{group_name}'")
        else:
            print(f"No analyzed data to save for {group_name} group.")

print("\n=========================================================")
print("        Analyzed Data Export Complete                    ")
print("=========================================================")

        Fetching Data and Calculating Indicators (150 days)

################## Group: ETF ##################

--- Processing data for SPY ---
--- Calculating indicators for SPY ---


/tmp/ipython-input-1726138086.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


Indicators calculated for SPY. DataFrame shape: (150, 35)

--- Processing data for QQQ ---
--- Calculating indicators for QQQ ---


/tmp/ipython-input-1726138086.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


Indicators calculated for QQQ. DataFrame shape: (150, 35)

--- Processing data for MAGS ---
--- Calculating indicators for MAGS ---
Indicators calculated for MAGS. DataFrame shape: (150, 35)

--- Processing data for IBIT ---


...

--- Calculating indicators for GOOGL ---
Indicators calculated for GOOGL. DataFrame shape: (150, 35)

--- Processing data for META ---
--- Calculating indicators for META ---
Indicators calculated for META. DataFrame shape: (150, 35)

--- Processing data for AMZN ---


/tmp/ipython-input-1726138086.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)
/tmp/ipython-input-1726138086.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for AMZN ---
Indicators calculated for AMZN. DataFrame shape: (150, 35)

--- Processing data for NVDA ---
--- Calculating indicators for NVDA ---
Indicators calculated for NVDA. DataFrame shape: (150, 35)

--- Processing data for TSLA ---


/tmp/ipython-input-1726138086.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)
/tmp/ipython-input-1726138086.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="1y", progress=False)


--- Calculating indicators for TSLA ---
Indicators calculated for TSLA. DataFrame shape: (150, 35)

        Data Fetching and Indicator Calculation Complete 

        Saving Analyzed Data to analyzed_financial_data.xlsx         
Analyzed data for ETF saved to sheet 'ETF'
Analyzed data for STOCK saved to sheet 'STOCK'

        Analyzed Data Export Complete                    


In [67]:
import pandas as pd
from datetime import datetime

# Assuming analyzed_data dictionary is available from the previous cell execution
if 'analyzed_data' in locals() or 'analyzed_data' in globals():
    # --- Add Timestamp to filename (Date only) ---
    now = datetime.now()
    timestamp = now.strftime("%d_%m_%Y") # Changed format to DD_MM_YYYY
    output_filename = f"analyzed_financial_data_{timestamp}.xlsx"
    # --- End Add Timestamp ---


    with pd.ExcelWriter(output_filename) as writer:
        for group_name, group_data in analyzed_data.items():
            if group_data: # Check if there is data for the group
                # Combine all tickers within the group into a single DataFrame for saving
                # Add a 'Ticker' column to identify the data for each ticker
                combined_df = pd.concat(
                    [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
                )

                # --- Reorder columns to place 'Ticker' after the index (Date) ---
                # Get the list of columns
                cols = combined_df.columns.tolist()
                # Remove 'Ticker' from its current position
                cols.remove('Ticker')
                # Insert 'Ticker' at the desired position (index 0 is the Date index, so insert at index 0 for the first column after index)
                cols.insert(0, 'Ticker') # Insert at the beginning of the column list

                # Reindex the DataFrame with the new column order
                combined_df = combined_df[cols]
                # --- End Reorder columns ---


                # Save the combined DataFrame for the group to a sheet named after the group
                # Reset index so 'Date' becomes a regular column for easier reordering
                combined_df.reset_index().to_excel(writer, sheet_name=group_name, index=False)

                print(f"Analyzed data for {group_name} saved to sheet '{group_name}' in {output_filename}")
            else:
                print(f"No analyzed data to save for {group_name} group.")

    print("\n=========================================================")
    print("        Analyzed Data Export Complete                    ")
    print(f"        Data saved to {output_filename}                 ")
    print("=========================================================")

else:
    print("Error: 'analyzed_data' variable not found. Please run the previous cell to analyze the data.")

Analyzed data for ETF saved to sheet 'ETF' in analyzed_financial_data_28_09_2025.xlsx
Analyzed data for STOCK saved to sheet 'STOCK' in analyzed_financial_data_28_09_2025.xlsx

        Analyzed Data Export Complete                    
        Data saved to analyzed_financial_data_28_09_2025.xlsx                 


In [61]:
!pip install pandas_ta
import pandas_ta as ta
import pandas as pd

# Assuming fetched_data is available from previous execution
if 'fetched_data' in locals() or 'fetched_data' in globals():
    analyzed_data = {}

    print("=========================================================")
    print("        Calculating Technical Indicators                 ")
    print("=========================================================")

    for group_name, group_data in fetched_data.items():
        print(f"\n################## Group: {group_name} ##################")
        analyzed_data[group_name] = {}

        for ticker, df in group_data.items():
            if not df.empty:
                print(f"\n--- Calculating indicators for {ticker} ---")
                # Calculate RSI (14)
                df.ta.rsi(length=14, append=True)

                # Calculate ATR (14)
                df.ta.atr(length=14, append=True)

                # Calculate Moving Averages
                df.ta.sma(length=50, append=True)
                df.ta.sma(length=100, append=True)
                df.ta.sma(length=150, append=True)
                df.ta.sma(length=200, append=True)

                # Calculate MACD (12, 26, 9)
                df.ta.macd(fast=12, slow=26, signal=9, append=True)

                analyzed_data[group_name][ticker] = df
                print(f"Indicators calculated for {ticker}. DataFrame shape: {df.shape}")
            else:
                print(f"!!! Skipping indicators for {ticker} as no data was available.")

    print("\n=========================================================")
    print("        Technical Indicator Calculation Complete         ")
    print("=========================================================")

    # You can now access the DataFrames with indicators in the 'analyzed_data' dictionary
    # For example, to see the data for SPY with new columns:
    # display(analyzed_data['ETF']['SPY'].tail())

else:
    print("Error: 'fetched_data' variable not found. Please run the previous cell to fetch the data.")

        Calculating Technical Indicators                 

################## Group: ETF ##################

--- Calculating indicators for SPY ---
Indicators calculated for SPY. DataFrame shape: (150, 13)

--- Calculating indicators for QQQ ---
Indicators calculated for QQQ. DataFrame shape: (150, 13)

--- Calculating indicators for MAGS ---
Indicators calculated for MAGS. DataFrame shape: (150, 13)

--- Calculating indicators for IBIT ---
Indicators calculated for IBIT. DataFrame shape: (150, 13)

--- Calculating indicators for ETHA ---
Indicators calculated for ETHA. DataFrame shape: (150, 13)

################## Group: STOCK ##################

--- Calculating indicators for AAPL ---
Indicators calculated for AAPL. DataFrame shape: (150, 13)

--- Calculating indicators for MSFT ---
Indicators calculated for MSFT. DataFrame shape: (150, 13)

--- Calculating indicators for GOOGL ---
Indicators calculated for GOOGL. DataFrame shape: (150, 13)

--- Calculating indicators for META ---
I

In [62]:
import yfinance as yf
import pandas as pd

# --- 1. שלב ראשון: הגדרת הנכסים ---

ASSET_GROUPS = {
    # נכסי ETF
    "ETF": ['SPY', 'QQQ', 'MAGS', 'IBIT', 'ETHA'],
    # מניות
    "STOCK": ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN', 'NVDA', 'TSLA']
}

# --- 2. שלב שני: בחירת ימי המסחר (אינטראקטיבי מתוך הרשימה) ---

def select_trading_days():
    """
    מאפשר למשתמש לבחור את מספר ימי המסחר הרצוי מתוך הרשימה המוגדרת בלבד.
    In Colab, this will default to the maximum available days.
    """
    # ימי המסחר המוגדרים כפי שצוין במסמך
    AVAILABLE_DAYS = [50, 100, 150, 200, 300]
    DEFAULT_DAYS = AVAILABLE_DAYS[-1] # ברירת מחדל: 300 ימים

    print("\n=========================================================")
    print("  שלב שני: בחירת ימי מסחר אחרונים (היסטוריה)")
    print(f"  אפשרויות זמינות: {', '.join(map(str, AVAILABLE_DAYS))}")
    print("---------------------------------------------------------")
    print(f"  בסביבת Colab, נבחר אוטומטית ברירת מחדל: {DEFAULT_DAYS} ימים.")

    selected_days = DEFAULT_DAYS

    # Removed interactive input for Colab compatibility
    # try:
    #     user_input = input("אנא רשום את המספר הרצוי: ").strip()
    # except EOFError:
    #     user_input = ""
    #     print(f"(נבחר {DEFAULT_DAYS} ימים כברירת מחדל עקב סביבת הרצה.)")

    # if user_input != "":
    #     try:
    #         input_days = int(user_input)
    #         if input_days in AVAILABLE_DAYS:
    #             selected_days = input_days
    #         else:
    #             print(f"❌ המספר {input_days} אינו ברשימת האפשרויות. נבחר {DEFAULT_DAYS} ימים.")
    #     except ValueError:
    #         print(f"❌ קלט לא חוקי. נבחר {DEFAULT_DAYS} ימים.")

    print(f"✅ נבחרו {selected_days} ימי מסחר.")
    return selected_days

# --- 3. שלב שלישי: משיכת הנתונים הנדרשים (Open, Low, High, Close, Volume) ---

def get_base_data(ticker, days):
    """
    מושך נתונים היסטוריים (Open, Low, High, Close, Volume) עבור טיקר ומספר ימים נבחר.
    """
    try:
        # משיכה של שנתיים כדי להבטיח שיש 300 ימי מסחר
        df = yf.download(ticker, period="2y", progress=False)
        if df.empty:
            return pd.DataFrame()
    except Exception:
        return pd.DataFrame()

    # שמירת העמודות הנדרשות בלבד
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df_final = df[['Open', 'Low', 'High', 'Close', 'Volume']].tail(days)

    return df_final


# --- 4. לולאת ביצוע מרכזית ---

def run_stages_1_to_3():

    # שלב 2: בחירת ימי המסחר
    N_DAYS_SELECTED = select_trading_days()

    print("=========================================================")
    print(f"  יישום שלבים 1 ו-3: משיכת נתוני בסיס ({N_DAYS_SELECTED} ימים)")
    print("=========================================================")

    final_results = {} # Dictionary to store results

    for group_name, tickers in ASSET_GROUPS.items():
        print(f"\n################## טאב: {group_name} ##################")

        group_data = {} # Dictionary to store data for the current group
        for ticker in tickers:
            print(f"\n--- משיכת נתונים עבור {ticker} ---")

            # שלב 3: ביצוע משיכת הנתונים
            df_base = get_base_data(ticker, N_DAYS_SELECTED)

            if not df_base.empty:
                print(f"נתוני בסיס אחרונים (5 שורות) עבור {ticker}:")
                print(f"  (אורך הנתונים בפועל: {len(df_base)} ימים)")
                print(df_base.tail(5).to_markdown(floatfmt=".2f"))
                group_data[ticker] = df_base # Store the DataFrame in the group dictionary
            else:
                print(f"!!! דילוג: לא נמצאו נתונים עבור {ticker}.")

        final_results[group_name] = group_data # Store the group dictionary in the final results

    print("\n=========================================================")
    print("                 סיום יישום שלבים 1-3                     ")
    print("=========================================================")
    return final_results # Return the final results dictionary

# הרצת התוכנית
fetched_data = run_stages_1_to_3()


  שלב שני: בחירת ימי מסחר אחרונים (היסטוריה)
  אפשרויות זמינות: 50, 100, 150, 200, 300
---------------------------------------------------------
  בסביבת Colab, נבחר אוטומטית ברירת מחדל: 300 ימים.
✅ נבחרו 300 ימי מסחר.
  יישום שלבים 1 ו-3: משיכת נתוני בסיס (300 ימים)

################## טאב: ETF ##################

--- משיכת נתונים עבור SPY ---


/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור SPY:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-22 00:00:00 | 666.84 | 662.17 | 667.29 |  662.20 | 69452200.00 |
| 2025-09-23 00:00:00 | 663.21 | 661.98 | 667.34 |  666.72 | 81708900.00 |
| 2025-09-24 00:00:00 | 661.10 | 659.67 | 664.61 |  664.51 | 68082200.00 |
| 2025-09-25 00:00:00 | 658.05 | 654.41 | 659.41 |  657.94 | 89622100.00 |
| 2025-09-26 00:00:00 | 661.82 | 657.88 | 662.37 |  659.51 | 69142300.00 |

--- משיכת נתונים עבור QQQ ---
נתוני בסיס אחרונים (5 שורות) עבור QQQ:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-22 00:00:00 | 602.20 | 597.72 | 602.87 |  597.74 | 57154800.00 |
| 2025-09-23 00:00:00 | 598.20 | 596.98 | 602.57 |  602.37 | 64635500.00 |


/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)
/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור MAGS:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |   Low |   High |   Close |     Volume |
|:--------------------|-------:|------:|-------:|--------:|-----------:|
| 2025-09-22 00:00:00 |  65.58 | 65.12 |  65.68 |   65.18 | 2472800.00 |
| 2025-09-23 00:00:00 |  64.57 | 64.44 |  65.73 |   65.73 | 2660400.00 |
| 2025-09-24 00:00:00 |  64.64 | 64.17 |  64.99 |   64.93 | 1859600.00 |
| 2025-09-25 00:00:00 |  64.10 | 63.51 |  64.43 |   64.10 | 3056700.00 |
| 2025-09-26 00:00:00 |  64.45 | 63.72 |  64.58 |   64.24 | 2903700.00 |

--- משיכת נתונים עבור IBIT ---
נתוני בסיס אחרונים (5 שורות) עבור IBIT:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |   Low |   High |   Close |      Volume |
|:--------------------|-------:|------:|-------:|--------:|------------:|
| 2025-09-22 00:00:00 |  63.65 | 63.58 |  64.46 |   64.18 | 42504100.00 |
| 2025-09-23 00:00:00 |  63.41 | 63.33 |  64.39 |   64.16 | 40145000.00 |
| 2025-09-24 00

...

נתוני בסיס אחרונים (5 שורות) עבור AMZN:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |      Volume |
|:--------------------|-------:|-------:|-------:|--------:|------------:|
| 2025-09-22 00:00:00 | 227.63 | 227.51 | 230.57 |  230.56 | 45914500.00 |
| 2025-09-23 00:00:00 | 220.71 | 220.07 | 227.86 |  227.83 | 70956200.00 |
| 2025-09-24 00:00:00 | 220.21 | 219.45 | 224.56 |  224.15 | 49509000.00 |
| 2025-09-25 00:00:00 | 218.15 | 216.47 | 220.67 |  220.06 | 52226300.00 |
| 2025-09-26 00:00:00 | 219.78 | 218.02 | 221.05 |  219.08 | 41600900.00 |

--- משיכת נתונים עבור NVDA ---


/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור NVDA:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |       Volume |
|:--------------------|-------:|-------:|-------:|--------:|-------------:|
| 2025-09-22 00:00:00 | 183.61 | 174.71 | 184.55 |  175.30 | 269637000.00 |
| 2025-09-23 00:00:00 | 178.43 | 176.21 | 182.42 |  181.97 | 192559600.00 |
| 2025-09-24 00:00:00 | 176.97 | 175.40 | 179.78 |  179.77 | 143564100.00 |
| 2025-09-25 00:00:00 | 177.69 | 173.13 | 180.26 |  174.48 | 191586700.00 |
| 2025-09-26 00:00:00 | 178.19 | 174.93 | 179.77 |  178.17 | 148306600.00 |

--- משיכת נתונים עבור TSLA ---


/tmp/ipython-input-3888966216.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="2y", progress=False)


נתוני בסיס אחרונים (5 שורות) עבור TSLA:
  (אורך הנתונים בפועל: 300 ימים)
| Date                |   Open |    Low |   High |   Close |       Volume |
|:--------------------|-------:|-------:|-------:|--------:|-------------:|
| 2025-09-22 00:00:00 | 434.21 | 429.13 | 444.98 |  431.11 |  97108800.00 |
| 2025-09-23 00:00:00 | 425.85 | 423.72 | 440.97 |  439.88 |  83422700.00 |
| 2025-09-24 00:00:00 | 442.79 | 429.03 | 444.21 |  429.83 |  93133600.00 |
| 2025-09-25 00:00:00 | 423.39 | 419.08 | 435.35 |  435.24 |  96746400.00 |
| 2025-09-26 00:00:00 | 440.40 | 421.02 | 440.47 |  428.30 | 101332800.00 |

                 סיום יישום שלבים 1-3                     


In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
import pandas as pd
from datetime import datetime

# Assuming fetched_data dictionary is available from the previous cell execution
if 'fetched_data' in locals() or 'fetched_data' in globals():
    # --- Get Date Timestamp ---
    now = datetime.now()
    timestamp = now.strftime("%d_%m_%Y") # Changed format to DD_MM_YYYY
    # --- End Get Date Timestamp ---

    for group_name, group_data in fetched_data.items():
        if group_data: # Check if there is data for the group
            # Combine all tickers within the group into a single DataFrame for saving
            # Add a 'Ticker' column to identify the data for each ticker
            combined_df = pd.concat(
                [df.assign(Ticker=ticker) for ticker, df in group_data.items()]
            )

            # --- Reorder columns to place 'Ticker' after the index (Date) ---
            # Get the list of columns
            cols = combined_df.columns.tolist()
            # Remove 'Ticker' from its current position
            cols.remove('Ticker')
            # Insert 'Ticker' at the desired position (index 0 is the Date index, so insert at index 0 for the first column after index)
            cols.insert(0, 'Ticker') # Insert at the beginning of the column list

            # Reindex the DataFrame with the new column order
            combined_df = combined_df[cols]
            # --- End Reorder columns ---

            # Define the filename based on the group name and timestamp
            filename = f"{group_name}_{timestamp}.csv" # Added timestamp to filename

            # Save the combined DataFrame to a CSV file
            # Reset index so 'Date' becomes a regular column for easier reordering
            combined_df.reset_index().to_csv(filename, index=False)


            print(f"Full results for {group_name} saved to {filename}")
        else:
            print(f"No data to save for {group_name} group.")

else:
    print("Error: 'fetched_data' variable not found. Please run the previous cell to generate the results.")

Full results for ETF saved to ETF_28_09_2025.csv
Full results for STOCK saved to STOCK_28_09_2025.csv
